In [1]:
%pip install semantic-link-labs

StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 7, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.7/764.7 kB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 63.1 MB/s eta 0:00:00
  Attempting uninstall: azure-core
    Found existing installation: azure-core 1.30.2
    Not uninstalling azure-core at /home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages, outside environment /nfs4/pyenv-f3d1a4aa-958b-44c8-90b6-26144928c46b
    Can't uninstall 'azure-core'. No files were found to uninstall.
  Attempting uninstall: semantic-link-sempy
    Found existing installation: semantic-link-sempy 0.11.0
    Not uninstalling semantic-link-sempy at /home/trusted-service-user/

In [2]:
import sempy.fabric as fabric
import sempy_labs as sempy_labs
from sempy_labs import migration, report, directlake
from sempy_labs import lakehouse as lake
from sempy_labs.tom import connect_semantic_model
import pandas as pd
import pyspark.sql.functions
from delta.tables import DeltaTable
from pyspark.sql.functions import lit, current_timestamp
from datetime import datetime
from sempy.fabric.exceptions import FabricHTTPException
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType
import numpy as np
import os

StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 9, Finished, Available, Finished)

# code

In [3]:
def fnc_startUp(_Tablename: StringType, _LH_Name: StringType, _WS_ID: StringType):
    """
    This function will do all the initial setups for the Semantic link flow

    - Check for lakehouse and create if necessary

    - check for table in that lakehouse and truncate or drop 
    """

    fnc_SET_PATH_abs_LH(_WS_ID, _LH_Name)

    fnc_SET_PATH_LocalMount()

    # Truncate or drop the table to have a clear start to insert into  
    fnc_TableCheckStartOfRun(_Tablename, 'Truncate',_LH_Name)

StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 10, Finished, Available, Finished)

# set global variables

In [4]:
def fnc_SET_PATH_abs_LH(_WorkspaceID: StringType, _LH_Name: StringType):
    """
    This function will check if the specified lakehouse exists and create it if not
    """
    Items = fabric.list_items(workspace=_WorkspaceID)
    FileInWS = Items[(Items['Type'] == 'Lakehouse') & (Items['Display Name'] == _LH_Name)]
    if(FileInWS.empty):
        try:
            lh = fabric.create_lakehouse(display_name=_LH_Name, workspace=_WorkspaceID)
            print(f"Lakehouse {_LH_Name} with ID {lh} successfully created")
        except FabricHTTPException as exc:
            print(f"An error occurred: {exc}")
    # Mount the Lakehouse for direct file system access
    lakehouse = mssparkutils.lakehouse.get(_LH_Name)
    mssparkutils.fs.mount(lakehouse.get("properties").get("abfsPath"), f"/{_LH_Name}")
    lh_abfs_path = lakehouse.get("properties").get("abfsPath")
    os.environ["ABS_LAKE_PATH"] = lh_abfs_path

StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 11, Finished, Available, Finished)

In [5]:
def fnc_SET_PATH_LocalMount():
    mount_name = '/temp_mnt'
    base_path = os.getenv("ABS_LAKE_PATH")
    mssparkutils.fs.mount(base_path, mount_name)
    mount_points = mssparkutils.fs.mounts()
    local_path = next((mp["localPath"] for mp in mount_points if mp["mountPoint"] == mount_name), None)
    os.environ["Mount_LAKE_PATH"] = local_path

StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 12, Finished, Available, Finished)

# initial setups

In [6]:
def fnc_TableCheckStartOfRun(_Tablename, _TruncateOrDrop):
    """
    Parameters:

        _Tablename = the table we are going to use

        _TruncateOrDrop = we can drop or Truncate the table, options to provide (Truncate or Drop)
    """
    lh_abfs_path = os.getenv("Mount_LAKE_PATH")
    notebookname = f"{lh_abfs_path}/Tables/{_Tablename}"

    if spark.catalog.tableExists(notebookname):
        if(_TruncateOrDrop == 'Truncate'):
            spark.sql(f'TRUNCATE table {_Tablename}')
        if(_TruncateOrDrop == 'Delete'):
            spark.sql(f'DROP table {_Tablename}')


StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 13, Finished, Available, Finished)

In [7]:
def fnc_NotebookRefreshTableCheck():
    lh_abfs_path = os.getenv("ABS_LAKE_PATH")
    lh_local_path = os.getenv("Mount_LAKE_PATH")
    tablename = "refreshTimes_Notebooks"
    tables = os.listdir(lh_local_path + "/Tables")
    exists = False
    for t in tables:
        if(t == tablename):
            exists=True
    print(exists)
    #if spark.catalog.tableExists(f"{lh_abfs_path}/Tables/{tableName}"):
    if(exists):
        print('refreshTimes_Notebooks Existed')
    else:    
        schema = StructType([
            StructField("NotebookName", StringType(), False),
            StructField("StartTime", TimestampType(), True),
            StructField("EndTime", TimestampType(), True)
        ])
        df = spark.createDataFrame([], schema)
        notebookname = f"{lh_abfs_path}/Tables/refreshTimes_Notebooks"
        print(notebookname)
        df.write.format("delta").save(notebookname)
        #df.write.format("delta").saveAsTable(f"{lh_abfs_path}/Tables/refreshTimes_Notebooks") 
        #df.write.format("delta").saveAsTable(f"{lh_abfs_path}/Tables/{tablename}") 

StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 14, Finished, Available, Finished)

# functions to call

In [8]:
def fnc_PrepareColumns(_Columns):
    _Columns.columns = _Columns.columns.str.replace('[^a-zA-Z0-9]', '', regex=True)
    _Columns.columns = _Columns.columns.str.replace('[ ]', '', regex=True)
    return _Columns

StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 15, Finished, Available, Finished)

In [28]:
def fnc_LognotebookRefresh(workspace_id, _NotebookName: StringType, _StartTime: datetime, _EndTime: datetime):
    lh_abfs_path = os.getenv("ABS_LAKE_PATH")
    Table_Name = "refreshTimes_Notebooks"
    d = spark.createDataFrame([(_NotebookName, _StartTime, _EndTime)],["NotebookName", "StartTime", "EndTime"])
    print(d)
    #df = spark.createDataFrame(data=d)
    d.write.format("delta").mode("append").option("mergeSchema", "true").save(f"{lh_abfs_path}/Tables/{Table_Name}")

StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 35, Finished, Available, Finished)

In [12]:
def fnc_WriteToTable(_TableDataFrame, lh_abfs_path):
    """
    This function accepts a dataframe, changes the column names to something a lakehouse will accept and then save it
    """
    try:
        PreparedDataframe = fnc_PrepareColumns(_TableDataFrame)
        sparkdf = spark.createDataFrame(PreparedDataframe)
        sparkdf = sparkdf.withColumn('WSID', lit(Id))
        sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")
    except Exception as e:
        print(f"Error fetching Workspace objects for {Id}: {e}")
        

StatementMeta(, 9603b890-7acf-45e7-975c-c6cbd7da32c6, 19, Finished, Available, Finished)